# BART Ridership 2018 data processing

## Export the raw data

In [1]:
# Dependencies for data munging
import os
import re
import pandas as pd
import numpy as np
from functools import reduce
import itertools
from datetime import datetime
import calendar

# Dependencies for data loading to database
from sqlalchemy import create_engine
from config import password
import pymysql
pymysql.install_as_MySQLdb()

# Add an encoder for numpy.float64
# Resource: https://stackoverflow.com/a/52949184
pymysql.converters.encoders[np.float64] = pymysql.converters.escape_float
pymysql.converters.conversions = pymysql.converters.encoders.copy()
pymysql.converters.conversions.update(pymysql.converters.decoders)

# Dependencies for API data requests
import requests
import json

In [2]:
# List down the files
path = "Data/"

files = os.listdir(path) # show the files in the Data folder
r = re.compile("Ridership*") # filter the files for those whose names include Ridership
files2 = list(filter(r.match, files)) # create a list of files matching the criterion r
files2.sort() # sort the file by order of month (based on file name)
files2 # preview the files list

['Ridership_201801.xlsx',
 'Ridership_201802.xlsx',
 'Ridership_201803.xlsx',
 'Ridership_201804.xlsx',
 'Ridership_201805.xlsx',
 'Ridership_201806.xlsx',
 'Ridership_201807.xlsx',
 'Ridership_201808.xlsx',
 'Ridership_201809.xlsx',
 'Ridership_201810.xlsx',
 'Ridership_201811.xlsx',
 'Ridership_201812.xlsx']

In [3]:
# Create a list of dataframes arranged by month
df_list = [pd.read_excel(path + file) for file in files2]

# Create a list of year-month based on the file names
filenames = [filename[10:14] + "-" + filename[14:16] for filename in files2]

## Transform the raw data

In [4]:
# Create an empty list that will be populated by the data with converted column headers
df_list2 = []

# Create a for-loop that cleans each of the original dataframes
for df in df_list:
    
    # Assign the top row of the original dataframe as the column header
    df.columns = df.iloc[0] 
    
    # The dataframe values come from row 1 onwards, removing the last row ("total entries")
    df = df[1:-1] 
    
    # Remove the "Exits" column
    df = df.drop("Exits", axis = 1) 
    
    # Call the first column "Exit_Station"
    df = df.rename(columns = {np.nan: "Exit_Station"}) 
    
    # Add each df to the df_list2 list
    df_list2.append(df)

# Preview the list
df_list2[0].head()

,Exit_Station,RM,EN,EP,NB,BK,AS,MA,19.0,12.0,...,ED,NC,WP,SS,SB,SO,MB,WD,OA,WS
1,RM,12.7619,108.619,81.9048,64.9524,365.619,101.571,138.048,149.190476,180.095238,...,16.0476,2.19048,35,12.1905,18.9048,58.7143,26.6667,7.61905,12.3333,7.47619
2,EN,131.952,20.3333,91.2857,95.7619,825.857,139.381,261.619,471.476190,562.285714,...,27.0476,5.57143,26.5714,34.0952,42.619,113.619,71.1905,13.2857,30.9048,26.4762
3,EP,86.4286,81.381,12.8571,46.9048,659.19,83.0476,133.762,298.000000,336.333333,...,14.7143,3.66667,11.1905,10.2857,11.1429,60.3333,31.8095,7.47619,24.7143,11
4,NB,72.381,87.1429,51.0476,15.619,199.619,61.3333,91.1905,285.190476,249.333333,...,14.1429,3.28571,16.2381,7.85714,9.80952,91.381,35.8095,8,30.7619,16.5714
5,BK,405.619,897.095,667.81,208,35.619,360.762,361.905,562.285714,528.428571,...,109.714,49.1429,113.048,34.4286,39.2381,179.905,139.095,48.5238,73.5238,78.381


In [5]:
# Define the function "pivot" which unpivots the dataframe to three columns
# NB: list(df.columns[1:]) = create a list of column headers excluding "Exit_Station"
def unpivot(df):
    transformed_df = pd.melt(df, id_vars = ["Exit_Station"], 
                             value_vars = list(df.columns[1:]), 
                             var_name = "Entry_Station") # name for new column
    return transformed_df

In [6]:
# Create a new list of dataframes containing transformed data (exit, entry, avg values)
df_list3 = [unpivot(df) for df in df_list2]

In [7]:
# Create a new list
df_list4 = []

# Rename the "value" column using the ith item in the filenames list
for i in range(len(filenames)):
    value_renamed = df_list3[i].rename(columns = {"value": filenames[i]})
    df_list4.append(value_renamed)

# View the new dataframe list
df_list4

[     Exit_Station Entry_Station  2018-01
 0              RM            RM  12.7619
 1              EN            RM  131.952
 2              EP            RM  86.4286
 3              NB            RM   72.381
 4              BK            RM  405.619
 5              AS            RM  105.048
 6              MA            RM  154.143
 7              19            RM  162.905
 8              12            RM  203.238
 9              LM            RM  36.4762
 10             FV            RM  91.1429
 11             CL            RM  60.3333
 12             SL            RM   33.381
 13             BF            RM  20.5714
 14             HY            RM  41.4762
 15             SH            RM  18.6667
 16             UC            RM   17.381
 17             FM            RM  32.0476
 18             CN            RM  23.8571
 19             PH            RM  18.4762
 20             WC            RM  20.1429
 21             LF            RM  5.09524
 22             OR            RM  

In [8]:
# Merge the 12 dataframes based on exit and entry stations
merged = reduce(lambda left, right: pd.merge(left, right, 
                                             on = ["Exit_Station", "Entry_Station"],
                                             how = "outer"), df_list4)
merged.head()

,Exit_Station,Entry_Station,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
0,RM,RM,12.7619,16.0526,17,13.9048,15.3182,13.9048,15.2381,14.8261,20.1053,16.2273,16.1579,16.1667
1,EN,RM,131.952,135.211,126.136,121.714,122.591,122.476,119.381,120.696,120.053,121.682,106.632,110.333
2,EP,RM,86.4286,93,84.5909,85.9048,94.2273,86.9048,81.619,84.087,88.2632,93.2727,83.4737,84.0556
3,NB,RM,72.381,74.7368,69.0909,70.9048,74.3636,62.381,61.8571,66.8261,78.6842,79.0909,68.1053,61.5556
4,BK,RM,405.619,473.263,449.636,438.524,406.636,410.571,432.619,438.348,496.105,479.864,431.158,363


In [9]:
# Transform the wide dataframe to a long one
merged2 = pd.melt(merged, id_vars = ["Exit_Station", "Entry_Station"],
       value_vars = list(merged.columns[2:]), var_name = "Yr_Mo", value_name = "Avg_Weekday_Trips")
merged2.head()

,Exit_Station,Entry_Station,Yr_Mo,Avg_Weekday_Trips
0,RM,RM,2018-01,12.7619
1,EN,RM,2018-01,131.952
2,EP,RM,2018-01,86.4286
3,NB,RM,2018-01,72.381
4,BK,RM,2018-01,405.619


In [10]:
# Create functions that extract the year and the month from the Yr_Mo column 
# and place them in separate columns

def str2year(s):
    my_list = s.tolist()
    out = [datetime.strptime(x, '%Y-%m').year for x in my_list]
    return(out)

def str2month(s):
    my_list = s.tolist()
    out = [datetime.strptime(x, '%Y-%m').month for x in my_list]
    return(out)

# Create a new merged dataframe containing the year and the month columns
merged3 = merged2.assign(Year = str2year(merged2["Yr_Mo"]),
               Month = str2month(merged2["Yr_Mo"]))

In [11]:
# Convert month number to month name abbreviation
# Resource: https://stackoverflow.com/a/37625467
merged3["Month"] = merged3["Month"].apply(lambda x: calendar.month_abbr[x])

In [12]:
# Rearrange the column names
merged3 = merged3[["Exit_Station", "Entry_Station", "Yr_Mo",
                   "Year", "Month", "Avg_Weekday_Trips"]]

# Preview the dataframe
merged3.head()

,Exit_Station,Entry_Station,Yr_Mo,Year,Month,Avg_Weekday_Trips
0,RM,RM,2018-01,2018,Jan,12.7619
1,EN,RM,2018-01,2018,Jan,131.952
2,EP,RM,2018-01,2018,Jan,86.4286
3,NB,RM,2018-01,2018,Jan,72.381
4,BK,RM,2018-01,2018,Jan,405.619


In [13]:
ridership = merged3.reset_index()

In [14]:
ridership = ridership.rename(columns = {"index":"ID"})
ridership.head()

,ID,Exit_Station,Entry_Station,Yr_Mo,Year,Month,Avg_Weekday_Trips
0,0,RM,RM,2018-01,2018,Jan,12.7619
1,1,EN,RM,2018-01,2018,Jan,131.952
2,2,EP,RM,2018-01,2018,Jan,86.4286
3,3,NB,RM,2018-01,2018,Jan,72.381
4,4,BK,RM,2018-01,2018,Jan,405.619


In [15]:
decimals = 2
ridership['Avg_Weekday_Trips'] = ridership['Avg_Weekday_Trips'].apply(lambda x: round(x, decimals))
ridership.head()

,ID,Exit_Station,Entry_Station,Yr_Mo,Year,Month,Avg_Weekday_Trips
0,0,RM,RM,2018-01,2018,Jan,12.76
1,1,EN,RM,2018-01,2018,Jan,131.95
2,2,EP,RM,2018-01,2018,Jan,86.43
3,3,NB,RM,2018-01,2018,Jan,72.38
4,4,BK,RM,2018-01,2018,Jan,405.62


## Create dataframe containing metadata of the different stations

In [16]:
# Request information from the BART Stations API
response = requests.get("https://api.bart.gov/api/stn.aspx?cmd=stns&key=MW9S-E7SL-26DU-VV8V&json=y")

In [17]:
# Load the json file
station_api = json.loads(response.text)

# Extract the metadata about the stations as a dictionary
station_dict = station_api["root"]["stations"]["station"]

# Convert the dictionary into a dataframe
station_df = pd.DataFrame(station_dict)

# Reformat the names of the counties
counties = {"alameda": "Alameda", "sanfrancisco": "San Francisco", 
          "sanmateo": "San Mateo", "contracosta": "Contra Costa"}

station_df["county"] = station_df["county"].replace(counties)
station_df.head()

,abbr,address,city,county,gtfs_latitude,gtfs_longitude,name,state,zipcode
0,12TH,1245 Broadway,Oakland,Alameda,37.803768,-122.271450,12th St. Oakland City Center,CA,94612
1,16TH,2000 Mission Street,San Francisco,San Francisco,37.765062,-122.419694,16th St. Mission,CA,94110
2,19TH,1900 Broadway,Oakland,Alameda,37.808350,-122.268602,19th St. Oakland,CA,94612
3,24TH,2800 Mission Street,San Francisco,San Francisco,37.752470,-122.418143,24th St. Mission,CA,94110
4,ANTC,1600 Slatten Ranch Road,Antioch,Contra Costa,37.995388,-121.780420,Antioch,CA,94509


In [18]:
# Load the station name Excel sheet and reset the index
stations = pd.read_excel(path + "Station_Names.xls")
stations = stations.reset_index().drop(["index"], axis = 1)

# Rename the columns for brevity
stations.rename(columns = {"Station Name": "name", "Two-Letter Station Code": "abbr2"}, 
                inplace = True)

# Preview the dataframe
stations.head()

,abbr2,name
0,RM,Richmond
1,EN,El Cerrito Del Norte
2,EP,El Cerrito Plaza
3,NB,North Berkeley
4,BK,Berkeley


In [19]:
# Append new BART stations 
# Pittsburg Center and Antioch stations opened in May 2018
# Milpitas and Berryessa stations projected to open in 2019
list_of_series = [pd.Series(["AN", "Antioch"], index = stations.columns), 
                  pd.Series(["PC", "Pittsburg Center"], index = stations.columns), 
                  pd.Series(["ML", "Milpitas"], index = stations.columns),
                  pd.Series(["BE", "Berryessa"], index = stations.columns)]

stations2 = stations.append(list_of_series, ignore_index= True)
stations2.head()

,abbr2,name
0,RM,Richmond
1,EN,El Cerrito Del Norte
2,EP,El Cerrito Plaza
3,NB,North Berkeley
4,BK,Berkeley


In [20]:
# Create a dictionary containing station names in "stations" and in "stations_df"
di = {"12th Street / Oakland City Center": "12th St. Oakland City Center", 
      "16th Street Mission": "16th St. Mission", "Montgomery Street": "Montgomery St.",
      "24th Street Mission": "24th St. Mission", "Berkeley": "Downtown Berkeley",
      "19th Street Oakland": "19th St. Oakland", "Bayfair": "Bay Fair", 
      "Civic Center": "Civic Center/UN Plaza", "El Cerrito Del Norte": "El Cerrito del Norte", 
      "North Concord": "North Concord/Martinez", "Powell Street": "Powell St.",
      "Pleasant Hill": "Pleasant Hill/Contra Costa Centre", 
      "Warm Springs": "Warm Springs/South Fremont"}

In [21]:
# Rename the stations names with stations_df names to match the names during the df merge
stations2["name"] = stations2["name"].replace(di)
stations2.head()

,abbr2,name
0,RM,Richmond
1,EN,El Cerrito del Norte
2,EP,El Cerrito Plaza
3,NB,North Berkeley
4,BK,Downtown Berkeley


In [22]:
# Merge stations and station_df using "name"
metadata = pd.merge(station_df, stations2, on = "name", how = "outer")

# Create a column for the ID
metadata = metadata.reset_index()
metadata = metadata.rename(columns = {"index":"ID"})

# List of coordinate variables
coords = ["gtfs_latitude", "gtfs_longitude"]

# Convert lat and long from string to float
metadata[coords] = metadata[coords].apply(pd.to_numeric, errors = "coerce")
metadata.head()

,ID,abbr,address,city,county,gtfs_latitude,gtfs_longitude,name,state,zipcode,abbr2
0,0,12TH,1245 Broadway,Oakland,Alameda,37.803768,-122.271450,12th St. Oakland City Center,CA,94612,12
1,1,16TH,2000 Mission Street,San Francisco,San Francisco,37.765062,-122.419694,16th St. Mission,CA,94110,16
2,2,19TH,1900 Broadway,Oakland,Alameda,37.808350,-122.268602,19th St. Oakland,CA,94612,19
3,3,24TH,2800 Mission Street,San Francisco,San Francisco,37.752470,-122.418143,24th St. Mission,CA,94110,24
4,4,ANTC,1600 Slatten Ranch Road,Antioch,Contra Costa,37.995388,-121.780420,Antioch,CA,94509,AN


## Load the processed data in a MySQL database

In [23]:
# create a connection
conn = "root:{0}@localhost:3306/bart_db".format(password) # Password is in config.py
engine = create_engine(f"mysql://{conn}")

In [24]:
# Confirm presence of tables
engine.table_names()

/anaconda3/envs/PythonData/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


['metadata', 'ridership']

In [25]:
# Load dataframes into tables
ridership.to_sql(name='ridership', con=engine, if_exists='replace', index=False)
metadata.to_sql(name='metadata', con=engine, if_exists='replace', index=False)

In [26]:
# Read the table contents (for ridership)
pd.read_sql_query("select * from ridership", con = engine).head()

,ID,Exit_Station,Entry_Station,Yr_Mo,Year,Month,Avg_Weekday_Trips
0,0,RM,RM,2018-01,2018,Jan,12.76
1,1,EN,RM,2018-01,2018,Jan,131.95
2,2,EP,RM,2018-01,2018,Jan,86.43
3,3,NB,RM,2018-01,2018,Jan,72.38
4,4,BK,RM,2018-01,2018,Jan,405.62


In [27]:
# Read the table contents (for metadata)
pd.read_sql_query("select * from metadata", con = engine)

,ID,abbr,address,city,county,gtfs_latitude,gtfs_longitude,name,state,zipcode,abbr2
0,0,12TH,1245 Broadway,Oakland,Alameda,37.803768,-122.271450,12th St. Oakland City Center,CA,94612,12
1,1,16TH,2000 Mission Street,San Francisco,San Francisco,37.765062,-122.419694,16th St. Mission,CA,94110,16
2,2,19TH,1900 Broadway,Oakland,Alameda,37.808350,-122.268602,19th St. Oakland,CA,94612,19
3,3,24TH,2800 Mission Street,San Francisco,San Francisco,37.752470,-122.418143,24th St. Mission,CA,94110,24
4,4,ANTC,1600 Slatten Ranch Road,Antioch,Contra Costa,37.995388,-121.780420,Antioch,CA,94509,AN
5,5,ASHB,3100 Adeline Street,Berkeley,Alameda,37.852803,-122.270062,Ashby,CA,94703,AS
6,6,BALB,401 Geneva Avenue,San Francisco,San Francisco,37.721585,-122.447506,Balboa Park,CA,94112,BP
7,7,BAYF,15242 Hesperian Blvd.,San Leandro,Alameda,37.696924,-122.126514,Bay Fair,CA,94578,BF
8,8,CAST,3301 Norbridge Dr.,Castro Valley,Alameda,37.690746,-122.075602,Castro Valley,CA,94546,CV
9,9,CIVC,1150 Market Street,San Francisco,San Francisco,37.779732,-122.414123,Civic Center/UN Plaza,CA,94102,CC
